# Calculator Lexer - Parser

## 

| Sintactic rules | Semantics rules |
| :-: | :-: |
| sentence -> sentence exprID ; | Escribe(sentence.s exprID.s;) |
| sentence -> sentence ID;| $table[ID.lexval] = 0$ |
| sentence -> expr;| $Escribe(expr.s)$ |
| sentence -> $\epsilon$ | |
| exprID -> $exprID_1$ `ID` `=` $expr$ | $exprID.s = table[ID.lexval] = expr.s |
| exprID ->$\epsilon$ | |
| expr -> $expr_1$ `+` sum | $expr.s = expr_1.s + sum.s$ |
| expr -> $expr_1$ `-` sum | $expr.s = expr_1.s - sum.s$ |
| expr -> sum |  expr.s = sum.s|
| sum -> $sum_1 `*` fact$ | $sum.s = sum_1.s * fact.s$ |
| sum -> $sum_1 `/` fact$ | $sum.s = sum_1.s / fact.s$ |
| sum -> fact | sum.s = fact.s|
| fact -> $- fact_1$ | fact.s = $- fact_1.s $|
| fact -> ID| fact.s = $table[ID.lexval]$
| fact -> NUM | fact.s = NUM.lexval |
| fact -> `(` expr `)` | fact.s = expr.s |

In [1]:
from sly import Lexer, Parser
import pandas as pd
import sys

In [7]:
class CalcLexer(Lexer):
    # set of tokens
    tokens = {
        'ID', 'CNUM'
    }
    # set of literals
    literals = {
        '+', '-', '*', '/', '(', ')', ';'
    }

    # ignore
    ignore = ' \t'

    # tokens
    ID = r'[a-zA-Z_][a-zA-Z-0-9_]*'
    

    
    # special case 
    @_(r'\d+')
    def CNUM(self, t):
        t.value = int(t.value)
        return t

    def error(self, t):
        print('<-'*10, "Illegal character '{}' ".format(t.value[0], '->'*10))
        t.type = "Illegal"
        t.value = t.value[0]
        self.index += 1
        return t

In [3]:
data = pd.read_csv('assets\operators.csv')
data

,operators
0,4 + 4;
1,5 - 5;
2,(5 * 7) + 5 - 78 / 4 + (6 - 7 ) * 2;


In [4]:
lexer = CalcLexer()
sentences = pd.read_csv('assets\operators.csv').operators
pass_or_not = []
all_token_pass = True

for index, sentence in enumerate(sentences):
    print('-' * 80,"{} Lexically Testing sentence: '{}'".format(index, sentence),'-' * 80, sep='\n')
    for token in lexer.tokenize(sentence):
        print(" type = '{}', value = '{}'".format(token.type, token.value))
        if all_token_pass and 'Illegal' in token.type:
            all_token_pass = False
    
    pass_or_not.append('Pass') if all_token_pass else pass_or_not.append('FAIL')
    all_token_pass = True

data['Test'] = pass_or_not

--------------------------------------------------------------------------------
0 Lexically Testing sentence: '4 + 4;'
--------------------------------------------------------------------------------
 type = 'CNUM', value = '4'
 type = '+', value = '+'
 type = 'CNUM', value = '4'
 type = ';', value = ';'
--------------------------------------------------------------------------------
1 Lexically Testing sentence: '5 - 5;'
--------------------------------------------------------------------------------
 type = 'CNUM', value = '5'
 type = '-', value = '-'
 type = 'CNUM', value = '5'
 type = ';', value = ';'
--------------------------------------------------------------------------------
2 Lexically Testing sentence: '(5 * 7) + 5 - 78 / 4 + (6 - 7 ) * 2;'
--------------------------------------------------------------------------------
 type = '(', value = '('
 type = 'CNUM', value = '5'
 type = '*', value = '*'
 type = 'CNUM', value = '7'
 type = ')', value = ')'
 type = '+', value = '+'

## Parser

In [5]:
class CalcParser(Parser):
    tokens = CalcLexer.tokens
    lexval = {}

    @_('input expr ";"')
    def input(self, p):
        print("{} {} ;".format(p.input, p.expr))
    
    @_('empty')
    def input(self, p):
        return ''

    @_('')
    def empty(self, p):
        pass

    @_('expr "+" sum')
    def expr(self, p):
        return p.expr + p.sum
    @_('expr "-" sum')
    def expr(self, p):
        return p.expr - p.sum

    @_("sum")
    def expr(self, p):
        return p.sum
    
    @_('sum "*" prod')
    def sum(self, p):
        return p.sum * p.prod
    
    @_('sum "/" prod')
    def sum(self, p):
        if p.prod != 0:
            return p.sum / p.prod
        raise ZeroDivisionError
    
    @_('prod')
    def sum(self, p):
        return p.prod
    
    @_(' "-" prod')
    def prod(self, p ):
        return - p.prod
    
    @_('CNUM')
    def prod(self, p):
        return p.CNUM
    
    @_(' "(" expr ")" ')
    def prod(self, p):
        return p.expr

## Testing parser

In [6]:
if __name__ == '__main__':
    lexer = CalcLexer()
    parser = CalcParser()

    sentences = pd.read_csv('assets\operators.csv').operators

    for sentence in sentences:
        print("Sentence: '{}'".format(sentence))
        parser.parse(lexer.tokenize(sentence))
    


Sentence: '4 + 4;'
 8 ;
Sentence: '5 - 5;'
 0 ;
Sentence: '(5 * 7) + 5 - 78 / 4 + (6 - 7 ) * 2;'
 18.5 ;
